### Ingestion del archivo "movie_genre.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo JSON utilizando "DataFrameReader" de Spark

In [0]:
movie_genre_schema = "movieId INT, genreId INT"

In [0]:
movie_genre_df = spark.read \
    .schema(movie_genre_schema)\
    .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

#### Paso 3 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
movie_genre_renamed_df = movie_genre_df \
    .withColumnRenamed("movieId","movie_id") \
    .withColumnRenamed("genreId","genre_id") 

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
movie_genre_final_df = add_ingestion_date(movie_genre_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment"))) \
.withColumn("file_date", lit(v_file_date))

#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
display(movie_genre_final_df)

In [0]:
#overwrite_partition(movie_genre_final_df,"movie_silver","movie_genre","file_date")
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.genre_id = src.genre_id AND tgt.file_date = src.file_date'
merge_delta_lake(movie_genre_final_df,'movie_silver','movie_genre',silver_folder_path,merge_condition,'file_date')

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movie_genre
GROUP BY file_date;